# CMIP6 data

This notebook shows how to access CMIP6 data from the cloud

## Import modules and libraries

*First, let's make sure the Python env is correct to run this notebook*:

In [ ]:
import os, sys, urllib, tempfile
with tempfile.TemporaryDirectory() as tmpdirname:
    sys.path.append(tmpdirname)
    repo = "https://raw.githubusercontent.com/obidam/ds2-2023/main/"
    urllib.request.urlretrieve(os.path.join(repo, "utils.py"), 
                               os.path.join(tmpdirname, "utils.py"))
    from utils import check_up_env
    check_up_env()

In [ ]:
import sys
import gcsfs
import xarray as xr
import intake
import zarr
import pandas as pd
# print(gcsfs.__version__)
# print(xr.__version__)
# print(intake.__version__)
# print(zarr.__version__)

# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token='anon')

# 
xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

## Read the full CMIP6 catalog

In [ ]:
df_full = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_full.sample(10)

### Make a subset of it

In [ ]:
# df = df_full.query("activity_id=='CMIP' & table_id == 'Omon' & variable_id == 'thetao' & experiment_id == 'historical' & member_id == 'r1i1p1f1'")
df = df_full.query("activity_id=='CMIP' & table_id == 'Omon' & institution_id == 'CNRM-CERFACS' & experiment_id == 'historical'")
# df = df_full.query('institution_id == "CNRM-CERFACS" & member_id=="r1i1p1f2" & source_id=="CNRM-CM6-1"')

# df = df_full.query("activity_id=='CMIP' & table_id == 'Omon' & variable_id == 'thetao' & experiment_id == 'abrupt-4xCO2'")

# df = df.query("source_id=='CNRM-CM6-1-HR' & variable_id=='thetao'") # Horizontal resolution up to 1/4 deg
# df = df.query("source_id=='CNRM-ESM2-1' & variable_id=='thetao'") # Horizontal resolution up to 1deg
df = df.query("source_id=='CNRM-ESM2-1' & (variable_id=='thetao' | variable_id=='so')") # Horizontal resolution up to 1deg

# df = df.sort_values('version')
df = df.sort_values('member_id')
df

## Read some data (1 row of the catalog)

In [ ]:
def open_cmip6(df_row):
    # get the path to zarr store
    zstore = df.zstore.values[-1]
    print(zstore)
    
    # create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

    # open it using xarray and zarr
    ds = xr.open_zarr(mapper, consolidated=True)
    print("Size of this dataset:", ds.nbytes/1e9,"Gb")

    return ds

ds = open_cmip6(df.iloc[0])
ds

### Play with it

In [ ]:
# Compute size of the full df selection:
total_size = 0 # Gb
for index, row in df.iterrows():
    ds = open_cmip6(row)
    total_size += ds.nbytes/1e9
print("Size of the selection of datasets:", total_size, "Gb")    

In [ ]:
sst = ds['thetao'].sel(lev=0, method='nearest')
sst

In [ ]:
sst.sel(time='1978-05-28T12:00:00', method='nearest').plot()

In [ ]:
sst.where(sst['lat']>=0).where(sst['lon']>=360-275).sel(time='1978-05-28T12:00:00', method='nearest').plot(xlim=[0, 120], ylim=[140, 270])

### Horizontal resolution of the grid

In [ ]:
ds['lat'].isel(x=0).diff('y').plot()

In [ ]:
ds['lon'].isel(y=0).diff('x').plot()